In [1]:
import numpy as np
import pandas as pd
import datetime
from time import time
import math
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from calendar import monthrange

In [60]:
dam_hourly = pd.read_csv('../output/dam_hourly.csv')
dam_hourly=dam_hourly.set_index(['month','hour'])
dam_hourly.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 49728 entries, (1, 0) to (9, 23)
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         49728 non-null  object 
 1   price        49728 non-null  float64
 2   volume       49728 non-null  float64
 3   year         49728 non-null  int64  
 4   day          49728 non-null  int64  
 5   day_of_week  49728 non-null  int64  
 6   value        49728 non-null  float64
 7   cat          49728 non-null  object 
dtypes: float64(3), int64(3), object(2)
memory usage: 3.1+ MB


In [61]:
pv_p50 = pd.read_csv('../output/pv/25mw_p50.csv').melt(id_vars='Month', var_name='hour', value_name='yield').rename(columns={"Month": "month"})
pv_p50=pv_p50.astype({'hour': 'int32'})
pv_p50 = pv_p50.set_index(['month','hour'])
pv_p50.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 288 entries, (1, 0) to (12, 23)
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   yield   288 non-null    float64
dtypes: float64(1)
memory usage: 3.2 KB


In [62]:
pv_p50_months = pv_p50.groupby(['month']).sum()
pv_p50_months

,yield
month,
1,6238.990660
2,5418.229563
3,6366.563226
4,6215.297129
5,6036.580021
6,5620.607755
7,6148.405108
8,6352.798784
9,6017.373585


In [63]:
pv_p50['yield_prop']=pv_p50.apply(lambda x: x['yield'] / pv_p50_months.loc[x.name[0]],axis=1)
pv_p50

,,yield,yield_prop
month,hour,,
1,0,0.0,0.0
2,0,0.0,0.0
3,0,0.0,0.0
4,0,0.0,0.0
5,0,0.0,0.0
...,...,...,...
8,23,0.0,0.0
9,23,0.0,0.0
10,23,0.0,0.0


In [64]:
dam_hourly=dam_hourly.join(pv_p50)
dam_hourly

date  price  volume  year  day  day_of_week     value  cat  \
month hour                                                                     
1     0     2016-01-01  40.10   477.5  2016    1            4  19147.75  off   
      0     2016-01-02  43.95   479.0  2016    2            5  21052.05  off   
      0     2016-01-03  44.81   476.0  2016    3            6  21329.56  off   
      0     2016-01-04  47.06   476.0  2016    4            0  22400.56  off   
      0     2016-01-05  42.16   406.0  2016    5            1  17116.96  off   
...                ...    ...     ...   ...  ...          ...       ...  ...   
12    23    2020-12-27  50.52   215.0  2020   27            6  10861.80  off   
      23    2020-12-28  50.59   179.0  2020   28            0   9055.61  off   
      23    2020-12-29  50.15   161.0  2020   29            1   8074.15  off   
      23    2020-12-30  50.09   110.0  2020   30            2   5509.90  off   
      23    2020-12-31  50.14   136.0  2020   31            3   6819.04  off   

            yield  yield_prop  
month hour                     
1     0       0.0         0.0  
      0       0.0         0.0  
      0       0.0         0.0  
      0       0.0         0.0  
      0       0.0         0.0  
...           ...         ...  
12    23      0.0         0.0  
      23      0.0         0.0  
      23      0.0         0.0  
      23      0.0         0.0  
      23      0.0         0.0  

[49728 rows x 10 columns]

In [65]:
dam_hourly['yield_prop_price']=dam_hourly['yield_prop'] * dam_hourly['price'] 

In [66]:
dam_hourly

date  price  volume  year  day  day_of_week     value  cat  \
month hour                                                                     
1     0     2016-01-01  40.10   477.5  2016    1            4  19147.75  off   
      0     2016-01-02  43.95   479.0  2016    2            5  21052.05  off   
      0     2016-01-03  44.81   476.0  2016    3            6  21329.56  off   
      0     2016-01-04  47.06   476.0  2016    4            0  22400.56  off   
      0     2016-01-05  42.16   406.0  2016    5            1  17116.96  off   
...                ...    ...     ...   ...  ...          ...       ...  ...   
12    23    2020-12-27  50.52   215.0  2020   27            6  10861.80  off   
      23    2020-12-28  50.59   179.0  2020   28            0   9055.61  off   
      23    2020-12-29  50.15   161.0  2020   29            1   8074.15  off   
      23    2020-12-30  50.09   110.0  2020   30            2   5509.90  off   
      23    2020-12-31  50.14   136.0  2020   31            3   6819.04  off   

            yield  yield_prop  yield_prop_price  
month hour                                       
1     0       0.0         0.0               0.0  
      0       0.0         0.0               0.0  
      0       0.0         0.0               0.0  
      0       0.0         0.0               0.0  
      0       0.0         0.0               0.0  
...           ...         ...               ...  
12    23      0.0         0.0               0.0  
      23      0.0         0.0               0.0  
      23      0.0         0.0               0.0  
      23      0.0         0.0               0.0  
      23      0.0         0.0               0.0  

[49728 rows x 11 columns]

In [70]:
daily=dam_hourly.reset_index().groupby(['date','year','month','day']).sum()[['yield_prop_price']].reset_index()
daily=daily.rename(columns={"yield_prop_price": "a", "B": "c"})

,date,year,month,day,yield_prop_price
0,2016-01-01,2016,1,1,48.155306
1,2016-01-02,2016,1,2,54.139077
2,2016-01-03,2016,1,3,53.148297
3,2016-01-04,2016,1,4,89.312707
4,2016-01-05,2016,1,5,97.067854
...,...,...,...,...,...
2095,2021-09-26,2021,9,26,36.988696
2096,2021-09-27,2021,9,27,119.677238
2097,2021-09-28,2021,9,28,118.903275
2098,2021-09-29,2021,9,29,115.447913
